In [35]:
import pandas as pd
import numpy as np

In [36]:
cleanData = pd.read_csv('DBtrainrides.csv')
cleanData.head()

,ID,line,path,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check
0,1573967790757085557-2407072312-14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time
1,349781417030375472-2407080017-1,18,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN,on_time,on_time
2,7157250219775883918-2407072120-25,1,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,8000406,4,Aachen-Rothe Erde,Nordrhein-Westfalen,Aachen,52066,6.116475,50.770202,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time
3,349781417030375472-2407080017-2,18,Aachen Hbf,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,NaN,NaN,0,0,NaN,on_time,on_time
4,1983158592123451570-2407080010-3,33,Herzogenrath|Kohlscheid,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,NaN,on_time,on_time


CleanUp time columns like: arrival_plan, departure_plan, arrival_change and departure_change

In [37]:
time_cols = ['arrival_plan', 'departure_plan', 'arrival_change', 'departure_change']
cleanData[time_cols].isna().sum()
print((cleanData[time_cols].isna().mean() * 100).round(2))
cleanData[time_cols] = cleanData[time_cols].apply(pd.to_datetime, errors='coerce')
cleanData[time_cols].dtypes

arrival_plan        10.25
departure_plan       0.00
arrival_change      23.07
departure_change    16.49
dtype: float64


arrival_plan        datetime64[ns]
departure_plan      datetime64[ns]
arrival_change      datetime64[ns]
departure_change    datetime64[ns]
dtype: object

CleanUp path, to check which station is the first and is the last and how many stations are in one path.

In [38]:

cleanData['path_clean'] = (
    cleanData['path']
    .astype(str)
    .str.strip()
    .str.replace(r'\s*\|\s*', '|', regex=True)  
    .str.replace(r'\|{2,}', '|', regex=True)    
)

cleanData.loc[
    cleanData['path'].isna() | cleanData['path_clean'].isin(['nan', 'None', '']),
    'path_clean'
] = np.nan

def count_stations(x):
    if not isinstance(x, str) or x.lower() == 'nan':
        return 1
    parts = [p for p in x.split('|') if p.strip() not in ["", "nan", "none"]]
    
    return len(parts) + 1

cleanData['station_count'] = cleanData['path_clean'].apply(count_stations)

cleanData['first_station'] = cleanData['path_clean'].str.split('|').str[0]
cleanData['last_station']  = cleanData['path_clean'].str.split('|').str[-1]

cleanData[['station', 'path_clean', 'station_count', 'first_station', 'last_station']].head()


,station,path_clean,station_count,first_station,last_station
0,Aachen Hbf,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,14,Stolberg(Rheinl)Hbf Gl.44,Aachen Schanz
1,Aachen Hbf,NaN,1,NaN,NaN
2,Aachen-Rothe Erde,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,25,Hamm(Westf)Hbf,Stolberg(Rheinl)Hbf
3,Aachen West,Aachen Hbf,2,Aachen Hbf,Aachen Hbf
4,Aachen West,Herzogenrath|Kohlscheid,3,Herzogenrath,Kohlscheid


**Question 1** 

Is it possible to extract information from the ID column? Does the ID column stay constant during the one entire train line?

In [39]:
cleanData['ID_fixed'] = cleanData['ID'].astype(str).str.replace(r"^-", "", regex=True) # I think the minus is useless, maybe from a hash value 


cleanData[['ID_part1', 'ID_part2', 'ID_part3']] = cleanData['ID_fixed'].str.split('-', n=2, expand=True)

In [40]:
cleanData['ID_part3'] = pd.to_numeric(cleanData['ID_part3'], errors='coerce')
match_ratio = (cleanData['station_count'] == cleanData['ID_part3']).mean() * 100
print(f"same value station_count and ID_part3: {match_ratio:.2f}%")

same value station_count and ID_part3: 100.00%


In [41]:
cleanData['ID_part2'] = cleanData['ID_part2'].astype(str).str.zfill(10)

In [42]:
cleanData['ID_part2_time'] = pd.to_datetime(cleanData['ID_part2'], format='%y%m%d%H%M', errors='coerce')

In [43]:
cleanData[['arrival_plan', 'departure_plan', 'arrival_change', 'departure_change','ID_part2_time']].head(10)

,arrival_plan,departure_plan,arrival_change,departure_change,ID_part2_time
0,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-07 23:12:00
1,NaT,2024-07-08 00:17:00,NaT,NaT,2024-07-08 00:17:00
2,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-07 21:20:00
3,2024-07-08 00:20:00,2024-07-08 00:21:00,NaT,NaT,2024-07-08 00:17:00
4,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:10:00
5,2024-07-08 00:30:00,2024-07-08 00:31:00,2024-07-08 00:30:00,2024-07-08 00:31:00,2024-07-08 00:23:00
6,2024-07-08 00:58:00,2024-07-08 00:58:00,NaT,NaT,2024-07-07 23:57:00
7,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-07 23:07:00
8,NaT,2024-07-08 00:37:00,NaT,2024-07-08 00:37:00,2024-07-08 00:37:00
9,2024-07-08 00:27:00,2024-07-08 00:27:00,2024-07-08 01:16:00,2024-07-08 01:17:00,2024-07-08 00:13:00


**Answer Question01:**

- ID_part_1: unique identifier for journey
- ID_part_2: Time of snapshot 
- ID_part_3: number of train stations (previous stops + current station)

**Qustion 2**

What’s the relationship between path, eva_nr and station? Can a stop be on the path when it’s still the station?

In [44]:
n_eva = cleanData['eva_nr'].nunique()
n_pairs = cleanData[['eva_nr', 'station']].drop_duplicates().shape[0]

print(f"Unique eva_nr: {n_eva}")
print(f"Unique eva_nr station pairs: {n_pairs}")

if n_eva == n_pairs:
    print("Each eva_nr is associated with exactly one station.")
else:
    print("Some eva_nr are associated with multiple stations.")


Unique eva_nr: 1996
Unique eva_nr station pairs: 1996
Each eva_nr is associated with exactly one station.


In [45]:
import unicodedata

def normalize(s):
    if not isinstance(s, str):
        return ""
    s = unicodedata.normalize('NFKC', s)
    return s.strip().lower()

def station_in_path_exact(row):
    if not isinstance(row['path_clean'], str) or row['path_clean'].lower() in ['nan', 'none', '']:
        return False
    station_norm = normalize(row['station'])
    path_stations = [normalize(p) for p in row['path_clean'].split('|')]
    return station_norm in path_stations

mask = cleanData.apply(station_in_path_exact, axis=1)

share_in_path = mask.mean() * 100

print(f"Percentage of stops where the normalized station name appears in its own path: {share_in_path:.5f}%")

print(f"\nCount of matches: {mask.sum()} out of {len(cleanData)} rows")
display(cleanData.loc[mask, ['station', 'path_clean']].head(10))


Percentage of stops where the normalized station name appears in its own path: 0.00141%

Count of matches: 29 out of 2061357 rows


,station,path_clean
9352,Köln Messe/Deutz,Dortmund Hbf|Herne|Gelsenkirchen Hbf|Essen Hbf...
9868,Köln Messe/Deutz,Köln/Bonn Flughafen|Köln Messe/Deutz|Köln Hbf
275775,Bruchsal,Bruchsal|Bruchsal Sportzentrum|Bruchsal Am Man...
299367,Bruchsal,Bruchsal|Bruchsal Sportzentrum|Bruchsal Am Man...
316862,Köln Messe/Deutz,Dortmund Hbf|Bochum Hbf|Wattenscheid|Essen Hbf...
317390,Köln Messe/Deutz,Köln/Bonn Flughafen|Köln Messe/Deutz|Köln Hbf
388724,Frankenthal Süd,Grünstadt|Kirchheim(Weinstr)|Herxheim am Berg|...
584558,Bruchsal,Bruchsal|Bruchsal Sportzentrum|Bruchsal Am Man...
608428,Bruchsal,Bruchsal|Bruchsal Sportzentrum|Bruchsal Am Man...
628847,Köln Messe/Deutz,Dortmund Hbf|Bochum Hbf|Wattenscheid|Essen Hbf...


**Question 3**

When does “city” change — is city the destination, the nearest city, the town name?

In [46]:
import unicodedata

def norm(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = unicodedata.normalize('NFKC', s)
    return s.strip().lower()

unique_pairs = cleanData[['station', 'city']].drop_duplicates()

total_unique_stations = unique_pairs['station'].nunique()
print(f"Total unique stations in dataset: {total_unique_stations}")

city_counts = (
    unique_pairs
    .assign(city_norm=lambda df: df['city'].apply(norm))
    .groupby('city_norm')['station']
    .nunique()
    .sort_values(ascending=False)
)

sum_city_stations = city_counts.sum()

print(f"\nSum of all station counts across cities: {sum_city_stations}")

if sum_city_stations == total_unique_stations:
    print("Each station belongs to exactly one city (1:1 relationship confirmed).")
else:
    diff = total_unique_stations - sum_city_stations
    print(f"Inconsistency detected: difference of {diff} stations between totals.")

print("\nTop 20 cities with the most stations:\n")
print(city_counts.head(20))

city_counts_df = city_counts.reset_index().rename(columns={
    'city_norm': 'city',
    'station': 'number_of_stations'
})

Total unique stations in dataset: 1996

Sum of all station counts across cities: 1996
Each station belongs to exactly one city (1:1 relationship confirmed).

Top 20 cities with the most stations:

city_norm
berlin               128
hamburg               54
münchen               42
frankfurt am main     29
düsseldorf            24
köln                  24
dresden               17
stuttgart             17
leipzig               16
dortmund              16
essen                 16
nürnberg              14
bremen                11
duisburg              11
hannover              10
rostock                9
mannheim               9
offenbach (main)       7
saarbrücken            7
potsdam                7
Name: station, dtype: int64


**Answer:**

City changes when a train leaves one administrative area and enters another.
It is the town name where the station is located, not the destination or nearest big city.

**Question 4**
- Find examples and outliers etc. in the columns `arrival_plan` and `departure_plan`,  `arrival_change` and `departure_change`,


In [47]:
cleanData[time_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061357 entries, 0 to 2061356
Data columns (total 4 columns):
 #   Column            Dtype         
---  ------            -----         
 0   arrival_plan      datetime64[ns]
 1   departure_plan    datetime64[ns]
 2   arrival_change    datetime64[ns]
 3   departure_change  datetime64[ns]
dtypes: datetime64[ns](4)
memory usage: 62.9 MB


In [48]:
mask_invalid = cleanData['departure_plan'] < cleanData['arrival_plan']
print(f"Invalid time order in {mask_invalid} rows")

Invalid time order in 0          False
1          False
2          False
3          False
4          False
           ...  
2061352    False
2061353    False
2061354    False
2061355    False
2061356    False
Length: 2061357, dtype: bool rows


In [49]:
delay_cols = ['arrival_delay_m', 'departure_delay_m']

summary = cleanData[delay_cols].describe(percentiles=[.01, .05, .5, .95, .99])
print("Delay Summary (in minutes):")
display(summary)

print("\nMissing values per column:")
print(cleanData[delay_cols].isna().sum())

outliers = cleanData.loc[
    (cleanData['arrival_delay_m'] > 120) | (cleanData['departure_delay_m'] > 120),
    ['station', 'arrival_delay_m', 'departure_delay_m']
]
print(f"\nFound {len(outliers)} extreme delays (>120 minutes):")
display(outliers.head(10))


Delay Summary (in minutes):


,arrival_delay_m,departure_delay_m
count,2.061357e+06,2.061357e+06
mean,1.176581e+00,1.223574e+00
std,3.407859e+00,3.418300e+00
min,0.000000e+00,0.000000e+00
1%,0.000000e+00,0.000000e+00
5%,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00
95%,6.000000e+00,6.000000e+00
99%,1.600000e+01,1.600000e+01
max,1.590000e+02,1.590000e+02



Missing values per column:
arrival_delay_m      0
departure_delay_m    0
dtype: int64

Found 11 extreme delays (>120 minutes):


,station,arrival_delay_m,departure_delay_m
214807,Gelsenkirchen Hbf,133,132
220433,Münster (Westf) Hbf,134,134
230281,Duisburg Hbf,157,156
230412,Düsseldorf Hbf,157,157
231097,Essen Hbf,159,159
331405,München Ost,136,135
333439,Rosenheim,140,137
881705,Doberlug-Kirchhain,133,133
882500,Elsterwerda,0,134
902612,Berlin Ostkreuz,132,132


**Question 05**

- Check whether `arrival_delay_m`  and `departure_delay_m` columns match up with the `plan`  and `change` columns by running some checking function.


In [50]:

check_cols = ['arrival_plan', 'departure_plan', 'arrival_change', 'departure_change']

cleanData[check_cols] = cleanData[check_cols].apply(pd.to_datetime, errors='coerce')

cleanData['arrival_delay_calc'] = (cleanData['arrival_change'] - cleanData['arrival_plan']).dt.total_seconds() / 60
cleanData['departure_delay_calc'] = (cleanData['departure_change'] - cleanData['departure_plan']).dt.total_seconds() / 60

cleanData['arrival_delay_diff'] = cleanData['arrival_delay_m'] - cleanData['arrival_delay_calc']
cleanData['departure_delay_diff'] = cleanData['departure_delay_m'] - cleanData['departure_delay_calc']

diff_summary = cleanData[['arrival_delay_diff', 'departure_delay_diff']].describe()
display(diff_summary)

threshold = 1.0
mismatch = cleanData.loc[
    (cleanData['arrival_delay_diff'].abs() > threshold) |
    (cleanData['departure_delay_diff'].abs() > threshold),
    ['station', 'arrival_plan', 'arrival_change', 'arrival_delay_m', 'arrival_delay_calc', 'arrival_delay_diff',
     'departure_plan', 'departure_change', 'departure_delay_m', 'departure_delay_calc', 'departure_delay_diff']
]

print(f"Found {len(mismatch)} rows with delay mismatch > {threshold} minute(s).")
display(mismatch.head(10))


,arrival_delay_diff,departure_delay_diff
count,1585720.0,1721431.0
mean,0.0,0.0
std,0.0,0.0
min,0.0,0.0
25%,0.0,0.0
50%,0.0,0.0
75%,0.0,0.0
max,0.0,0.0


Found 0 rows with delay mismatch > 1.0 minute(s).


,station,arrival_plan,arrival_change,arrival_delay_m,arrival_delay_calc,arrival_delay_diff,departure_plan,departure_change,departure_delay_m,departure_delay_calc,departure_delay_diff


In [51]:
cleanData.drop(columns=['arrival_delay_calc', 'departure_delay_calc', 'arrival_delay_diff', 'departure_delay_diff'], inplace=True)

In [52]:
cleanData.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061357 entries, 0 to 2061356
Data columns (total 29 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   line                   object        
 2   path                   object        
 3   eva_nr                 int64         
 4   category               int64         
 5   station                object        
 6   state                  object        
 7   city                   object        
 8   zip                    int64         
 9   long                   float64       
 10  lat                    float64       
 11  arrival_plan           datetime64[ns]
 12  departure_plan         datetime64[ns]
 13  arrival_change         datetime64[ns]
 14  departure_change       datetime64[ns]
 15  arrival_delay_m        int64         
 16  departure_delay_m      int64         
 17  info                   object        
 18  arrival_delay_check   

**Answer**

the columns arrival_delay_m and departure delays match up with the plan and change

**Question 06**

- Check whether the `arrival_delay_check` and `departure_delay_check` have an *accidental* mistake — is there a row where the delay is over 6 minutes, but check returns `on_time` ? Use pandas to check all rows.

In [53]:
arrival_mistake = cleanData[
    (cleanData['arrival_delay_m'] > 6) & (cleanData['arrival_delay_check'].str.lower() == 'on_time')
]

departure_mistake = cleanData[
    (cleanData['departure_delay_m'] > 6) & (cleanData['departure_delay_check'].str.lower() == 'on_time')
]

print(f"Arrival mistakes found: {len(arrival_mistake)}")
print(f"Departure mistakes found: {len(departure_mistake)}")

Arrival mistakes found: 0
Departure mistakes found: 0


The arrival_delay_check and departure_delay_check are correct

**Question 7**

- Check all the info values (perhaps by using `.unique` ) — do they contain something useful? If yes, then explain; if no, could we consider removing that?


In [54]:
unique_info = cleanData['info'].dropna().unique()
print(f"Number of unique info values: {len(unique_info)}")

info_counts = cleanData['info'].value_counts(dropna=False).head(20)
display(info_counts)

non_empty = cleanData['info'].notna().sum()
total = len(cleanData)
print(f"\nRows with info value: {non_empty}/{total} ({non_empty/total*100:.2f}%)")

sample_info = cleanData[
    cleanData['info'].notna() &
    ~cleanData['info'].isin(['', 'None', 'nan', '-', 'NaN'])
]['info'].unique()[:10]

print("\nExample distinct info values:")
for val in sample_info:
    print(f"• {val}")


Number of unique info values: 7


info
NaN                                   1416016
Information                            244033
Störung                                116325
Bauarbeiten                             96301
Information. (Quelle: zuginfo.nrw)      78977
Bauarbeiten. (Quelle: zuginfo.nrw)      72555
Störung. (Quelle: zuginfo.nrw)          28744
Großstörung                              8406
Name: count, dtype: int64


Rows with info value: 645341/2061357 (31.31%)

Example distinct info values:
• Bauarbeiten. (Quelle: zuginfo.nrw)
• Information
• Information. (Quelle: zuginfo.nrw)
• Bauarbeiten
• Störung
• Großstörung
• Störung. (Quelle: zuginfo.nrw)


In [55]:
cleanData['info_clean'] = (
    cleanData['info']
    .str.replace(r'\. \(Quelle: zuginfo\.nrw\)', '', regex=True)
    .str.strip()
)

cleanData['info_clean'].value_counts()


info_clean
Information    323010
Bauarbeiten    168856
Störung        145069
Großstörung      8406
Name: count, dtype: int64

In [56]:
info_share = cleanData['info_clean'].value_counts(normalize=True) * 100
print(info_share)

info_clean
Information    50.052608
Bauarbeiten    26.165392
Störung        22.479433
Großstörung     1.302567
Name: proportion, dtype: float64


These values are informative and consistent, the column should definitely be kept for further analysis

**Question 8**

Check weird times — do trains also run at night? What happens to them at night?

In [57]:
cleanData['hour'] = cleanData['departure_plan'].dt.hour

hourly_counts = cleanData['hour'].value_counts().sort_index()
display(hourly_counts)

night = cleanData[cleanData['hour'].between(0, 4)]
share_night = len(night) / len(cleanData) * 100
print(f"Trains between 00:00–04:59: {len(night)} rows ({share_night:.2f}% of total)")

hourly_delay = (
    cleanData.groupby('hour')[['arrival_delay_m', 'departure_delay_m']]
    .mean()
    .rename(columns={'arrival_delay_m': 'avg_arrival_delay', 'departure_delay_m': 'avg_departure_delay'})
)
display(hourly_delay)


hour
0      49615
1      25361
2      11010
3       9645
4      34775
5      78902
6     102692
7     112599
8     109416
9     105135
10    102122
11    102314
12    103397
13    106730
14    107958
15    112288
16    116142
17    116490
18    113410
19    105587
20     97754
21     88845
22     80069
23     69101
Name: count, dtype: int64

Trains between 00:00–04:59: 130406 rows (6.33% of total)


,avg_arrival_delay,avg_departure_delay
hour,,
0,1.340502,1.373919
1,1.311857,1.328378
2,1.077748,1.101181
3,0.681182,0.735407
4,0.311517,0.367707
5,0.537439,0.586018
6,0.690258,0.739113
7,0.838595,0.880843
8,1.001736,1.044710


Trains do run at night, but only about 6 % of all services.
During 3-4 AM, the network is almost inactive, trains are rare and highly punctual.
Delays rise steadily throughout the day, peaking in the late afternoon and early evening.

In [58]:
cleanData.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2061357 entries, 0 to 2061356
Data columns (total 31 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   line                   object        
 2   path                   object        
 3   eva_nr                 int64         
 4   category               int64         
 5   station                object        
 6   state                  object        
 7   city                   object        
 8   zip                    int64         
 9   long                   float64       
 10  lat                    float64       
 11  arrival_plan           datetime64[ns]
 12  departure_plan         datetime64[ns]
 13  arrival_change         datetime64[ns]
 14  departure_change       datetime64[ns]
 15  arrival_delay_m        int64         
 16  departure_delay_m      int64         
 17  info                   object        
 18  arrival_delay_check   

**Task 1.25**

try to find the data points for a specific train ride. In other words, say that you picked line 18 that was supposed to arrive in Aachen West at 2024-07-08 00:20:00 — gather ALL data points that are related to that one train ride. This might just mean fetching all the lines with the same ID value and then sorting them by arrival time ascending.

In [61]:
example_train = cleanData[
    (cleanData['line'] == '18') &
    (cleanData['station'] == 'Aachen West') &
    (cleanData['arrival_plan'] == '2024-07-08 00:20:00')
]

if example_train.empty:
    print("No exact match found for this line/time.")
else:
    train_id = example_train.iloc[0]['ID']
    print(f"Train ID for this ride: {train_id}")

    full_ride = cleanData[cleanData['ID'] == train_id].copy()

    full_ride = full_ride.sort_values('arrival_plan', ascending=True)

    print(f"\nTrain {train_id} – Line 18 – full journey overview:")
    display(full_ride[['station', 'arrival_plan', 'departure_plan', 'arrival_delay_m', 'departure_delay_m', 'city']])


Train ID for this ride: 349781417030375472-2407080017-2

Train 349781417030375472-2407080017-2 – Line 18 – full journey overview:


,station,arrival_plan,departure_plan,arrival_delay_m,departure_delay_m,city
3,Aachen West,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,Aachen


In [64]:
full_line = cleanData[
    (cleanData['line'] == '18') &
    (cleanData['arrival_plan'].dt.date == pd.Timestamp('2024-07-08').date())
].sort_values('arrival_plan')
display(full_line[['station', 'arrival_plan', 'departure_plan', 'city', 'line']].head(20))


,station,arrival_plan,departure_plan,city,line
5869,Wendlingen (Neckar),2024-07-08 00:06:00,2024-07-08 00:06:00,Wendlingen am Neckar,18
4706,Plochingen,2024-07-08 00:13:00,2024-07-08 00:14:00,Plochingen,18
4908,Reutlingen Hbf,2024-07-08 00:13:00,2024-07-08 00:14:00,Reutlingen,18
136,Bad Friedrichshall Hbf,2024-07-08 00:14:00,2024-07-08 00:16:00,Bad Friedrichshall,18
3,Aachen West,2024-07-08 00:20:00,2024-07-08 00:21:00,Aachen,18
1609,Esslingen (Neckar),2024-07-08 00:20:00,2024-07-08 00:20:00,Esslingen am Neckar,18
4226,Neckarsulm,2024-07-08 00:20:00,2024-07-08 00:21:00,Neckarsulm,18
5398,Stuttgart-Bad Cannstatt,2024-07-08 00:27:00,2024-07-08 00:28:00,Stuttgart,18
3524,Ludwigsburg,2024-07-08 00:28:00,2024-07-08 00:29:00,Ludwigsburg,18
2667,Herzogenrath,2024-07-08 00:29:00,2024-07-08 00:30:00,Herzogenrath,18
